In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms as T
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import nrrd
import pandas as pd
import os
import sys

# sys.path.append("/mnt/raid/C1_ML_Analysis/source/autoencoder/src")
sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/")
sys.path.append("/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl/nets")

from nets import diffusion, spade, lotus, cut, layers, cut_G
from loaders import ultrasound_dataset as usd
import monai


In [3]:


batch_size = 1
dm = usd.CutDataModule(
    csv_train=["/mnt/raid/C1_ML_Analysis/cut_multi_dataset/c1_instance_table_sonosite_resampled_file_path_extract_frames_blind_sweeps_merged_train_train_balanced.parquet", 
        "/mnt/raid/C1_ML_Analysis/cut_multi_dataset/c2_instance_table_clarius_resampled_file_path_extract_frames_blind_sweeps_merged_train_train_balanced.parquet", 
        "/mnt/raid/C1_ML_Analysis/cut_multi_dataset/extract_frames_blind_sweeps_butterfly_merged_train_train_balanced.parquet", 
        "/mnt/raid/C1_ML_Analysis/cut_multi_dataset/extract_frames_blind_sweeps_voluson_merged_train_train_balanced.parquet"],
    csv_valid=["/mnt/raid/C1_ML_Analysis/cut_multi_dataset/c1_instance_table_sonosite_resampled_file_path_extract_frames_blind_sweeps_merged_train_test_balanced.parquet", 
        "/mnt/raid/C1_ML_Analysis/cut_multi_dataset/c2_instance_table_clarius_resampled_file_path_extract_frames_blind_sweeps_merged_train_test_balanced.parquet", 
        "/mnt/raid/C1_ML_Analysis/cut_multi_dataset/extract_frames_blind_sweeps_butterfly_merged_train_test_balanced.parquet", 
        "/mnt/raid/C1_ML_Analysis/cut_multi_dataset/extract_frames_blind_sweeps_voluson_merged_train_test_balanced.parquet"],
    csv_test=["/mnt/raid/C1_ML_Analysis/cut_multi_dataset/c1_instance_table_sonosite_resampled_file_path_extract_frames_blind_sweeps_merged_test_balanced.parquet", 
        "/mnt/raid/C1_ML_Analysis/cut_multi_dataset/c2_instance_table_clarius_resampled_file_path_extract_frames_blind_sweeps_merged_test_balanced.parquet", 
        "/mnt/raid/C1_ML_Analysis/cut_multi_dataset/extract_frames_blind_sweeps_butterfly_merged_test_balanced.parquet", 
        "/mnt/raid/C1_ML_Analysis/cut_multi_dataset/extract_frames_blind_sweeps_voluson_merged_test_balanced.parquet"],
    mount_point="/mnt/raid/C1_ML_Analysis/",
    batch_size=batch_size,
    num_workers=1,
    img_column="file_path",
    class_column="pred_class",
    prefetch_factor=2,
    num_samples_train=100,
    num_samples_val=100,
    num_samples_test=100,
    drop_last=False
    )
dm.setup()

In [4]:
test_dl = dm.test_dataloader()

In [5]:
from torchvision.utils import make_grid
import plotly.express as px

def plot_tensor_grid(tensor, nrow=4, title="Image Grid", scale=1.0):
    """
    Uses torchvision's make_grid to arrange images into a grid, then displays it with Plotly.

    Args:
        tensor: PyTorch tensor of shape [B, N, 1, H, W]
        nrow: Number of images per row in the grid
        title: Plot title
    """
    B, N, C, H, W = tensor.shape
    assert C == 1, "Only grayscale images (C=1) are supported."

    # Flatten batch and image dimensions
    tensor = tensor.view(B * N, C, H, W)

    # Create image grid
    grid = make_grid(tensor, nrow=nrow, padding=2, normalize=True, scale_each=True)

    # Convert to numpy for Plotly
    grid_np = grid.permute(1,2,0).cpu().numpy()  # shape [H, W] since it's grayscale

    # Determine scaled dimensions
    height, width, C = grid_np.shape
    display_height = int(height * scale)
    display_width = int(width * scale)

    # Plot with Plotly
    fig = px.imshow(grid_np, color_continuous_scale='gray')
    fig.update_layout(
        title=title,
        coloraxis_showscale=False,
        width=display_width,
        height=display_height
    )
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()

In [ ]:
model = cut.CutG.load_from_checkpoint("/mnt/raid/C1_ML_Analysis/train_output/Cut/allvslast/allvssonosite/v0.8/last.ckpt").eval().cuda()

In [24]:
batch = next(iter(test_dl))
X, X_labels, Y, Y_labels = batch
print(X.shape, Y.shape)
# plot_tensor_grid(torch.stack(X, dim=0), nrow=4, title='Image Grid')
px.imshow(X.squeeze(), facet_col=0, binary_string=True)

In [25]:

with torch.no_grad():
    out = model(X.cuda())
px.imshow(out.squeeze().cpu().numpy(), facet_col=0, color_continuous_scale='gray')

In [ ]:
# D_Y = monai.networks.nets.resnet18(n_input_channels=1, num_classes=1, spatial_dims=2)
# D_Y(torch.rand(1, 1, 256, 256)).shape

In [ ]:
# Use clusters 0, 4, 6, 7